# FizzBuzz in Qiskit

**Fizz buzz** is a group word game for children to teach them about division [1]. Someone chosen by lot starts counting from $1$ then each next player takes turns incerementing a previous number and saying the result except on occasion it should be replaced by
* *Fizz* if the result is divisible by $3$,
* *Buzz* if the result is divisible by $5$,
* *Fizz Buzz* if both occur simultaneously.

A wrong answer means the player is out of the game continued round by round. As a result there are a word chain likes "One, Two, *Fizz*, Four, *Buzz*, *Fizz*, Seven, Eight, ..." and a winner who had no mistakes.

**FizzBuzz** is also a common programming task to test basic skills. In this case a sequence from $1$ to $100$ should be a program output and could be obtained in many ways. So there are a lot of its implementations in different programming languages. Let's try to realize a simplified version of FizzBuzz in Qiskit using Grover's Search Algorithm (GSA) for multiple solutions.

The goal is to create a FizzBuzz sequence from $0$ to $15$ due to some reasons. All the numbers have a four bit representation, two auxiliary bits are enough to carry a condition check that is simplified in the range, and there are two FizzBuzz opening and closing the chain. Consider a binary string $$c_5c_4c_3c_2c_1c_0$$ where two most significant bits $c_5c_4$ keep some information about a number encoded in four least significant bits $c_3c_2c_1c_0$. Classical approaches are generally based on for-loop which body contains a divisibility test therefore the number of iterations $N$ equals to amount of elements. It corresponds to a linear time complexity $O\left(N\right)$ and may vary slightly for some implementations meanwhile quantum computing looks promising to improve that.

GSA is an unstructured search algorithm presented by Lov Grover in 1996 [2]. Instead of serial brute force it processes all data in parallel increasing a probability of desired outcomes in final distribution. Create a quantum circuit and perform $N=\frac\pi4\sqrt\frac N K$ iterations giving square root time complexity $O\left(\sqrt N\right)$. That means quadratic speedup so GSA provides quantum supremacy.

It is required only 6 qubits that is not be splitted into main and ancillary registers in a code but can be mentioned this way in description. 

## Imports

Since all required functions such as Grover Oracle or Inverse Quantum Fourier Transform (IQFT) are defined below it is enough to import some standard Qiskit modules and obviously numpy to get the $\pi$ value.

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, Aer, IBMQ, QuantumRegister, ClassicalRegister, execute
from qiskit.visualization import plot_histogram

## Initialization

At the beginning of circuit we applying Hadamard gate to each qubit. It gives a uniform superposition over all states in a main register and prepares ancilla qubits to phase adding. The latter is actually done by performing Quantum Fourier Transform to correct conversion of possible non-zero register value but that is not necessary at the initialization stage.

In [ ]:
def init(qc):
    qc.h(range(6))
    qc.barrier()

## Divisibility by 3

We have $3_{10}=11_2$ and it is very easy to check if a binary number satisfies the condition. Same as for decimal the algorithm takes an alternating digit sum. In other word we find the sums of its even and odd digits and compare them to each other. If they are equal or their difference is multiple of $11_2$ then the binary number is divisible by $11_2$. Note that in four bit register a maximum absolute value of the result equals to $10_2$. It means the second statement is always false that's why the divisibility test simplifies to $$c_3-c_2+c_1-c_0\equiv0$$

The left-hand side can be done in two ways: a bit adder with CNOT and CCNOT gates or a phase adder with CP gate and we choose the second one. It simplifies

In [ ]:
def div_by_3(qc):
    qc.cp(np.pi/2, 3, 4)
    qc.cz(3, 5)
    qc.cp(np.pi/2, 1, 4)
    qc.cz(1, 5)
    qc.cp(-np.pi/2, 2, 4)
    qc.cz(2, 5)
    qc.cp(-np.pi/2, 0, 4)
    qc.cz(0, 5)
    qc.barrier()

## Divisibility by 5

It seems a bit harder but very similar to the previous one. Now we have $3_{10}={0101}_2=11_4$ and each binary pair can be turned into a quaternary digit so we will compare the sums of even and odd pairs of digits. Just like before if they are equal or their difference is multiple of $0101_2$ then the binary number is divisible by $0101_2$. In four bit register there are only two pairs, the odd and the even, so we go straight to substraction skipping the addition. In this case a maximum absolute value of the difference could equal to $11_2$. The second statement is always false again and the divisibility test simplifies to $$c_3c_2-c_1c_0\equiv0$$

In [ ]:
def div_by_5(qc):
    qc.cp(np.pi/2, 2, 4)
    qc.cz(2, 5)
    qc.cz(3, 4)
    qc.cp(-np.pi/2, 0, 4)
    qc.cz(0, 5)
    qc.cz(1, 4)
    qc.barrier()

## Inverse Quantum Fourier Transform

After arithmetic calculations it's time to go back to the initial Z-basis and we perform IQFT for that.

In [ ]:
def iqft(qc):
    qc.h(4)
    qc.cp(-np.pi/2, 4, 5)
    qc.h(5)
    qc.swap(4, 5)
    qc.barrier()

## Oracle

Grover's Oracle marks 'good' states flipping their phase to the negative one so let's take a look what a 'good' state in our example is. Both tests we define above return a null string in ancilla register if and only if a number sutisfies the conditions. The register consists of two qubits and the best choice is the CZ-gate switches a phase for only one state out of four.

In [ ]:
def oracle(qc):
    qc.x(range(4, 6))
    qc.cz(4, 5)
    qc.x(range(4, 6))
    qc.barrier()

## Diffuser

In [ ]:
def diffuser(qc):
    qc.h(range(6))
    qc.x(range(6))
    qc.mcp(np.pi, [0, 1, 2, 3, 4], 5)
    qc.x(range(6))
    qc.h(range(6))

## Fizz

Now just put all functions together to build a first quantum circuit finds multiples of $3$.

In [ ]:
fizz = QuantumCircuit(6)
init(fizz)
div_by_3(fizz)
iqft(fizz)
oracle(fizz)
diffuser(fizz)
fizz.measure_all()
fizz.draw()

We get a main simulator of the Aer provider as a backend. It mimics a behavior of real quantum hardware systems those may not be available to everyone. By default the simulator is noise-free what is useful for education or debugging. But it is possible to execute the circuits on real quantum devices too with additional error correction.

In [ ]:
simulator = Aer.get_backend('aer_simulator')
job_fizz = execute(fizz, simulator, shots = 10000)
result_fizz = job_fizz.result()
counts_fizz = result_fizz.get_counts()
plot_histogram(counts_fizz, figsize = (18, 6))

## Buzz

In [ ]:
buzz = QuantumCircuit(6)
init(buzz)
div_by_5(buzz)
iqft(buzz)
oracle(buzz)
diffuser(buzz)
buzz.measure_all()
buzz.draw()

In [ ]:
job_buzz = execute(buzz, simulator, shots = 10000)
result_buzz = job_buzz.result()
counts_buzz = result_buzz.get_counts()
plot_histogram(counts_buzz, figsize = (18, 6))

## Ouptut

As a result of running the circuits we have obtained two dictionaries plotted above as histograms. It's clear that each high bar corresponds to some binary string with zeros in two MSBs (ancilla register) and required number in four LSBs (main register). This is an expected result because the strings describe good states amplified by GSA. Now they are obviously distinguish by counts from others (more than twice) but we still have only raw data so let's move on to creating an output.

The output we work on is a list of strings so create some pattern for the beginning. Then apply firstly 'Fizz' and secondly 'Buzz' to elements those indices are decoded from amplified strings main register. Other strings are still empty should be filled out their indices directly.

In [ ]:
output = [''] * 16
for key in counts_fizz.keys():
    if counts_fizz[key] > 500:
        output[int(key[2:], 2)] += 'Fizz'
for key in counts_buzz.keys():
    if counts_buzz[key] > 500:
        output[int(key[2:], 2)] += 'Buzz'
print(output)